# Project: Oil Well Profitability Prediction for OilyGiant

## Inspiration
This project is a capstone challenge that simulates working as a machine learning specialist in a high-stakes business environment. I will use real-world techniques to help OilyGiant make data-driven decisions on where to drill new oil wells. It’s a test of my ability to combine modeling, business logic, and statistical analysis, all independently.

## Project Goal
My goal is to determine which of three regions provides the highest profit potential and lowest financial risk for developing 200 new oil wells. I will use linear regression to predict reserve volumes, apply business constraints, and use bootstrapping to analyze profit uncertainty.

## Project Plan

### Step 1: Data Preparation
- Load the datasets: `geo_data_0.csv`, `geo_data_1.csv`, and `geo_data_2.csv`.
- Check for missing values, duplicates, or anomalies.
- Confirm consistent structure: columns `id`, `f0`, `f1`, `f2`, `product`.

### Step 2: Train and Evaluate Models
For each of the three datasets:
1. Split the data into training (60%), validation & test sets (%20).
2. Train a linear regression model.
3. Make predictions on the validation set.
4. Save the predictions and true `product` values.
5. Print:
   - Mean predicted reserve volume
   - Root Mean Squared Error (RMSE)
6. Summarize model accuracy for each region.

### Step 3: Prepare for Profit Estimation
- Define constants:
  - Budget: 100 million USD  
  - Wells to develop: 200  
  - Revenue per 1,000 barrels: 4,500 USD  
- Calculate break-even reserve volume per well.
- Compare this to the average predicted reserves per region.
- Summarize which regions meet the break-even threshold.

### Step 4: Estimate Profit
- Write a function that:
  - Selects the top 200 predicted wells.
  - Sums the actual `product` values for those wells.
  - Computes revenue and subtracts the budget.
- Run this function for each region.
- Identify the most profitable region based on initial model predictions.

### Step 5: Risk and Confidence Analysis (Bootstrapping)
For each region:
1. Draw 1,000 random samples of 500 wells each from the validation set.
2. For each sample:
   - Predict reserves
   - Select top 200
   - Calculate profit using the actual `product` values
3. Calculate:
   - Average profit  
   - 95% confidence interval  
   - Probability of loss (i.e., negative profit)
4. Disqualify regions with more than 2.5% probability of loss.
5. From the qualified regions, I will select the one with the highest average profit.

### Step 6: Final Recommendation
- Recommend a single region to develop based on:
  - Model accuracy  
  - Average profit  
  - Risk level  
  - Business constraints  
- Summarize the justification clearly and concisely.

# Importing Libraries and Dataset

#### Core libraries

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

##### Modeling

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.model_selection import train_test_split

##### Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error

##### Visualization 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

##### Datasets

In [ ]:
geo_data_0 = pd.read_csv("/datasets/geo_data_0.csv")

In [ ]:
geo_data_1 = pd.read_csv("/datasets/geo_data_1.csv")

In [ ]:
geo_data_2 = pd.read_csv("/datasets/geo_data_2.csv")

# Analyzing Data

Functions for section

In [ ]:
def analyze(df):
    display(df.head())
    info = df.info()
    duplicated_amount = df.duplicated().sum()
    missing_values = df.isna().sum()
    print(info)
    print(f"\nDuplicates: {duplicated_amount}\n")
    print(f"\nMissing Values: \n{missing_values}\n")
    print("Duplicate Ids:",df['id'].duplicated().sum(), "\n", df['id'].duplicated().head())

In [ ]:
def skewness_and_outliers(df, region_name = "Region"): 
    print(f"\nSkewness and Outliers for {region_name}\n")
    
    # Summary stats
    print('Descriptive Stats:\n', df['product'].describe())
    
    # Quantiles
    print('\nQuantiles:\n', df['product'].quantile([0.01, 0.25, 0.5, 0.75, 0.99]))
    
    # Two histograms side by side
    plt.figure(figsize = (14, 4))
    
    # Histogram
    plt.subplot(1, 2, 1)
    df['product'].hist(bins = 50)
    plt.title(f'{region_name} - Product Histogram')
    plt.xlabel('Product (thousand barrels)')
    plt.ylabel('Frequency')
    plt.grid(True)
    
    # Density histogram
    plt.subplot(1, 2, 2)
    df['product'].hist(bins = 50, density = True)
    plt.title(f'({region_name}) - Product Density Histogram')
    plt.xlabel('Product (thousand barrels)')
    plt.ylabel('Density')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Boxplot
    plt.figure(figsize = (8, 2))
    sns.boxplot(x = df['product'])
    plt.title(f'Boxplot of Product Values ({region_name})')
    plt.show

In [ ]:
analyze(geo_data_0)

The dataset has 5 columns with 100,000 entries, no duplicate rows, no missing values, and all datatypes per column are correct. However 10 duplicate ids have been found. 

Corrections:
    
    - Remove rows with duplicate well ids, keep higher production wells.

In [ ]:
# Removing duplicate well ids

geo_data_0.sort_values('product', ascending=False, inplace=True)
geo_data_0.drop_duplicates(subset='id', keep='first', inplace=True)

In [ ]:
# Confirming duplicate ids have been removed
geo_data_0['id'].duplicated().sum()

Checking for skewness and outliers.

In [ ]:
skewness_and_outliers(geo_data_0, region_name = 'geo_data_0')

Results for geo_data_0:

- Distribution shape: Fairly symmetric and balanced — mean (92.5), median (91.85)

- Spread: Moderate variability with reserves ranging from 0 to 185k barrels

- Outliers: Some very low producing wells near 0, but no extreme high outliers

- Risk insight: The bottom 1% of wells produce less than 5k barrels, potentially unprofitable

In [ ]:
analyze(geo_data_1)

The dataset has 5 columns with 100,000 entries, no duplicate rows, no missing values, and all datatypes per column are correct. However 10 duplicate ids have been found. 

Corrections:
    
    - Remove rows with duplicate well ids, keep higher production well.

In [ ]:
# Removing duplicate well ids
geo_data_1.sort_values('product', ascending=False, inplace=True)
geo_data_1.drop_duplicates(subset='id', keep='first', inplace=True)

In [ ]:
# Confirming duplicate ids are removed
geo_data_1['id'].duplicated().sum()

In [ ]:
skewness_and_outliers(geo_data_1, region_name = 'geo_data_1')

Results for geo_data_1:

- Average reserves are low: Mean = 68.83k barrels, which is significantly below Region 0 (92.5).

- Even distribution: Histogram shows evenly spaced peaks.

- 1% of wells are dry: Bottom 1% of wells produce 0 barrels, a risk for investment.

- Symmetry and stability: Median (57.09) and mean (68.83) are close, with no visual outliers on boxplot.

In [ ]:
analyze(geo_data_2)

The dataset has 5 columns with 100,000 entries, no duplicate rows, no missing values, and all datatypes per column are correct. However 10 duplicate ids have been found. 

Corrections:
    
    - Remove rows with duplicate well ids, keep higher production well.

In [ ]:
# Removing duplicate wells
geo_data_2.sort_values('product', ascending=False, inplace=True)
geo_data_2.drop_duplicates(subset='id', keep='first', inplace=True)

In [ ]:
# Confirming duplicate ids are removed
geo_data_2['id'].duplicated().sum()

In [ ]:
skewness_and_outliers(geo_data_2, region_name = 'geo_data_2')

Results for geo_data_2:

- Highest average yield of all regions: 95k barrels

- Balanced distribution — no skew, no outliers

- Top 25% of wells are highly productive (>130k)

- Bottom 1% risk still exists (5k barrels), but similar to other regions

- Standard deviation is typical (44.75), so the spread is acceptable for modeling and business planning

##### Data Analysis Conclusion:

Region 2 offers the best overall yield and production range, without introducing more risk than other regions. It is likely the most profitable candidate.

# Train and Evaluate Models

Function for section.

Although only a train/validation split is required, a separate test set was added to verify model generalization and ensure robustness in RMSE estimation.

Why RMSE over MSE?
RMSE (Root Mean Squared Error) is used instead of MSE because it preserves the original units of the target variable, in this case, thousands of barrels. This makes the error more interpretable, if RMSE is 38, it means the model’s predictions deviate by about 38,000 barrels on average. MSE, being squared, would make the interpretation difficult.

In [ ]:
def train_validate_model(data, target_column = 'product', data_number = None):
    # Features and target
    x = data.drop(['id', target_column], axis = 1) # Drop id because its a name tag which serves no predictive value
    y = data[target_column]
    
    # First split 60% / 40%
    x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size = 0.4, random_state = 12345) 
    # Second split 20% / 20%
    x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size = 0.5, random_state = 12345)
    
    # Train model
    model = LinearRegression() 
    model.fit(x_train, y_train)
    predictions_train = model.predict(x_train)
    predictions_valid = model.predict(x_valid)
    predictions_test = model.predict(x_test)
    
    # Evaluation 
    rmse_train = mean_squared_error(y_train, predictions_train, squared = False)
    rmse_valid = mean_squared_error(y_valid, predictions_valid, squared = False)
    rmse_test = mean_squared_error(y_test, predictions_test, squared = False)
    mean_prediction_train = predictions_train.mean()
    mean_prediction_valid = predictions_valid.mean()
    mean_prediction_test = predictions_test.mean()
    average_prediction_error = (rmse_test / y_test.mean()) * 100
    
    
    return pd.DataFrame([{
        'model': model,
        'data_number': data_number,
        'average_prediction_error': average_prediction_error,
        'rmse_train': rmse_train,
        'rmse_valid': rmse_valid,
        'rmse_test': rmse_test,
        'rmse_gap': rmse_valid - rmse_test,
        'rmse_train_valid_gap': rmse_train - rmse_valid,
        'rmse_train_test_gap': rmse_train - rmse_test,
        'mean_prediction_train': mean_prediction_train,
        'mean_prediction_valid': mean_prediction_valid,
        'mean_prediction_test': mean_prediction_test,
        'mean_prediction_valid_gap': mean_prediction_valid - y_valid.mean(),
        'mean_prediction_test_gap': mean_prediction_test - y_test.mean(),
        'mean_prediction_train_valid_gap': mean_prediction_train - y_valid.mean(),
        'mean_prediction_train_test_gap': mean_prediction_test - y_test.mean(),
        'x_train': x_train,
        'y_train': y_train,
        'x_valid': x_valid,
        'y_valid': y_valid,
        'x_test': x_test,
        'y_test': y_test,
        'predictions_valid': predictions_valid
    }])

##### Region 0

In [ ]:
result_0 = train_validate_model(geo_data_0, data_number = 0)
result_0

###### Interpretation

- The model is very stable across train, validation, and test sets.

- No signs of overfitting, RMSEs are nearly identical across all splits.

- Mean predictions are unbiased, aligned with actual target means.

- Average prediction error 41%

- This model is suited for profit modeling, it can be trusted to generalize to new data.

Next, the same process will be repeated for Regions 1 and 2 to complete the model evaluation phase.

##### Region 1

In [ ]:
result_1 = train_validate_model(geo_data_1, data_number = 1)
pd.set_option('display.max_columns', None)
display(result_1)

###### Interpretation

With a 1.30% average prediction error and low RMSE, we can say the model is highly accurate on average and predicts around 69k barrels per well with very little bias or variance.

This performance will be compared to Regions 0 and 2 before selecting a region for profit evaluation.

##### Region 2

In [ ]:
result_2 = train_validate_model(geo_data_2, data_number = 2)
result_2

###### Interpretation

Region 2's model is well-calibrated and consistent across train, validation, and test sets, with very low prediction bias.
However, its average prediction error is high (42%), indicating that although predictions are centered correctly, their spread is wide. This makes individual predictions less reliable, similar to Region 0 in terms of RMSE, but with higher average reserves.

All three regional models will now be compared using visualizations and a summary table to help guide profit-focused decision-making.

In [ ]:
# Sample summary data from Regions 0, 1, and 2
summary_df = pd.DataFrame({
    'Region': ['Region 0', 'Region 1', 'Region 2'],
    'Mean Product': [92.5, 68.8, 95.0],
    'Average Prediction Error (%)': [40.6, 1.3, 42.5]
})

plt.figure(figsize = (14, 4))


# Plot Average Barrels
plt.subplot(1, 2, 1)
plt.bar(summary_df['Region'], summary_df['Mean Product'], label = 'Average Barrels Produced')
plt.title('Average Barrels Produced Across Regions')
plt.ylabel('Product')
plt.legend()
plt.grid(axis='y')

# Plot average prediction error
plt.subplot(1, 2, 2)
plt.bar(summary_df['Region'], summary_df['Average Prediction Error (%)'], color='salmon')
plt.title('Average Prediction Error (%) by Region')
plt.ylabel('Error Percentage')
plt.grid(axis='y')

plt.tight_layout()
plt.show

## Model Training and Evaluation Summary

In this section, I trained and evaluated a linear regression model for each of the three regions using a **60/20/20** split for training, validation, and testing. The goal was to predict oil reserves (`product`) and assess how well the model generalizes to new data using **RMSE** and **average prediction error** as metrics.

### Key Insights:

| Region    | Avg. Reserves (Barrels) | RMSE (Valid) | Prediction Error (%) | Notes |
|-----------|--------------------------|---------------|------------------------|-------|
| Region 0  | 92.5                     | 37.66         | 41.05%                 | Stable model, moderate reserves, high error |
| Region 1  | 68.8                     | 0.89          | 1.30%                  | Excellent accuracy, low variance |
| Region 2  | 95.0                     | 40.07         | 42.14%                 | High reserves, but high prediction spread |

### Visual Comparison:
- **Left Chart:** Region 2 has the highest average predicted reserves, followed closely by Region 0.
- **Right Chart:** Region 1 outperforms in accuracy by a wide margin, with less than 2.5% error, while Regions 0 and 2 have errors exceeding 40%.

### Conclusion:
- **Region 1** is most reliable for consistent predictions and minimal risk.
- **Region 2** may offer the highest returns, but with less confidence in individual predictions.
- All models were stable across train/valid/test, indicating no signs of overfitting.

# Prepare for Profit Estimation

### Constants:
- Budget: 100,000,000 USD  
- Revenue per 1,000 barrels: 4,500 USD 
- Wells to develop: 200 

In [ ]:
budget = 100_000_000
rev_per_1k_barrels = 4_500
wells_to_dev = 200

Cost per well.

In [ ]:
cost_per_well = budget / wells_to_dev
cost_per_well

Break even volume per well.

In [ ]:
break_even_vol = cost_per_well / rev_per_1k_barrels
print(f"Break even volume per well: {break_even_vol:.2f} thousand barrels")

Checking to see if average predicted reserves exceed break even in all regions:

| Region    | Avg. Reserves (Barrels) | 
|-----------|--------------------------|
| Region 0  | 92.5                     | 
| Region 1  | 68.8                     | 
| Region 2  | 95.0                     |

Although none of the regions exceed the break-even volume of 111.11 thousand barrels per well on average, the investment strategy targets only the top 200 wells per region. This makes the average less relevant, as profits depend on the high-performing subset.

The bootstrapping analysis will test whether the selected top 200 wells (from each random sample of 500) can consistently produce enough total volume to exceed the break-even threshold.

# Estimate Profit

Function for section

In [ ]:
def calculate_profit(predictions, actuals, count = wells_to_dev, budget = budget, revenue_per_1k = rev_per_1k_barrels):
    # Step 1: Get indices of top 200 predictions
    top_200_indices = predictions.argsort()[-200:]
    # Step 2: Use those indices to get the actual reserve
    top_200_actuals = actuals.iloc[top_200_indices]
    # Step 3: Compute total revenue = sum(actual reserves) × 4500
    revenue = top_200_actuals.sum() * revenue_per_1k
    # Step 4: Subtract $10M budget
    profit = revenue - budget
    # Step 5: Return profit
    return profit

##### Region 0 Profit Calculation

In [ ]:
row_0 = result_0.iloc[0]
profit_0 = calculate_profit(row_0['predictions_valid'], row_0['y_valid'])
print(f"Estimated profit for Region 0: ${profit_0:.2f}")

###### Region 0 Profit Calculation Summary
**Estimated profit for Region 0: $29,891,464.19**

**Region 0 Profit Calculation Summary**

If the top 200 wells are selected based on the model's predictions, the actual reserves from those wells would generate approximately **29.9 million in net profit**, after subtracting the full **100 million budget**.

This result is based on validation set outcomes and assumes an effective well selection strategy driven by the model. While the region's average reserves are below the break-even threshold, targeted selection of high-performing wells enables a profitable outcome.

##### Region 1 Profit Calculation

In [ ]:
row_1 = result_1.iloc[0]
profit_1 = calculate_profit(row_1['predictions_valid'], row_1['y_valid'])
print(f"Estimated profit for Region 1: ${profit_1:.2f}")

###### Region 1 Profit Calculation Summary
**Estimated profit for Region 1: $24,150,866.97**

**Region 1 Profit Calculation Summary**

Based on actual reserves from the top 200 wells selected by the model, Region 1 is projected to generate approximately **24.15 million in net profit** after accounting for the full **100 million investment budget**.

Despite having lower average reserves (68.8k barrels per well), Region 1 achieves strong profitability due to the **model’s exceptional accuracy (1.30% error)** and **very low RMSE**, which enables more reliable well selection. This makes Region 1 a compelling option under the targeted investment strategy.

##### Region 2 Profit Calculation

In [ ]:
row_2 = result_2.iloc[0]
profit_2 = calculate_profit(row_2['predictions_valid'], row_2['y_valid'])
print(f"Estimated profit for Region 2: ${profit_2:.2f}")

###### Region 2 Profit Calculation Summary
**Estimated profit for Region 2: $25,658,945.30**

**Region 2 Profit Calculation Summary**

Investing in Region 2’s top 200 wells, as selected by the model, is projected to yield approximately **25.66 million in net profit** after subtracting the full **100 million investment budget**.

While Region 2 has the highest average reserves (95.0k barrels), the model’s predictions come with greater variance and higher error (42.14%). Nevertheless, the selection of top-performing wells still results in a solid return, making Region 2 a competitive, slightly riskier, candidate.

In [ ]:
# Bar chart to compare profits

# Region profit results
profits = {
    'Region 0': profit_0,
    'Region 1': profit_1,
    'Region 2': profit_2
}

# Convert to dataframe for plottting
profit_df = pd.DataFrame(list(profits.items()), columns = ['Region', 'Estimated Profit'])

# Plotting
plt.figure(figsize = (8, 5))
plt.bar(profit_df['Region'], profit_df['Estimated Profit'], color = ['skyblue', 'lightgreen', 'salmon'])
plt.title('Estimated Profit From Top 200 Wells per Region')
plt.ylabel('Profit ($)')
plt.grid(axis = 'y')
plt.tight_layout()

## Estimate Profit Summary

- Region 0 delivers the highest estimated profit despite having a relatively high prediction error.

- Region 1 is the most accurate and consistent, with the lowest RMSE and error, but slightly less profit.

- Region 2 is a middle ground, great reserves and decent profit, but also the highest uncertainty.

Based on validation results, Region 0 offers the highest expected profit from developing the top 200 predicted wells, followed by Region 2 and Region 1. However, Region 0 and 2 come with significantly higher prediction error (40%+), whereas Region 1’s model is highly accurate and consistent. Profitability must therefore be balanced against model confidence and risk, which will be addressed in the next step via bootstrapping.

# Risk and Confidence Analysis (Bootstrapping)

Bootstrapping function

In [ ]:
def bootstrap_profit(model, features, targets, iterations = 1000, wells_sample = 500, top_n = 200):
    profits = []

    for _ in range(iterations):
        # 1. Sample indices (with replacement)
        sample_indices = np.random.choice(features.index, size = wells_sample, replace = False)
        
        # 2. Get the sampled features and targets
        sampled_features = features.loc[sample_indices]
        sampled_targets = targets.loc[sample_indices]

        # 3. Predict reserves for the sample
        predicted = model.predict(sampled_features)

        # 4. Select indices of top 200 predictions
        top_indices = predicted.argsort()[-top_n:]

        # 5. Get actual values for those top wells
        actual_top = sampled_targets.iloc[top_indices]

        # 6. Calculate profit
        total_revenue = actual_top.sum() * rev_per_1k_barrels
        profit = actual_top.sum() * 4500 - 100_000_000
        profits.append(profit)

    return profits

##### Region 0

In [ ]:
region_0_bs = bootstrap_profit(model = row_0['model'], features = row_0['x_valid'], targets = row_0['y_valid'])

In [ ]:
avg_profit_0 = np.mean(region_0_bs) # Average profit

conf_int_0 = np.percentile(region_0_bs, [2.5, 97.5]) # 95% confidence interval

loss_probability_0 = np.mean(np.array(region_0_bs) < 0) * 100 # Probability of loss if profit < 0

In [ ]:
print(f"Region 0 \nAverage Profit: {avg_profit_0:.2f}\nConfidence Interval: {conf_int_0}\nLoss Probabilty: {loss_probability_0}")

###### Region 0 Bootstrapping Summary

- **Average Profit:** 4,369,246.86  
- **95% Confidence Interval:** -656,366 to 9,367,585  
- **Probability of Loss:** 4.4% *(Exceeds the 2.5% risk threshold)*

**Interpretation:**

While Region 0's model identifies profitable wells and delivers a **positive average return**, the **risk of loss (4.4%) exceeds the acceptable threshold** defined by the business requirement (≤2.5%).

**Confidence Interval Interpretation – Region 0**

The 95% confidence interval for Region 0 ranges from **-656,366 to 9,367,585**, indicating substantial uncertainty in profit outcomes. This means that in some simulated investment scenarios, the project could even result in a **net loss**.

Such a wide interval reflects **high variability** in well performance and model predictions. The spread suggests that even though the average profit is positive, the investment results are **not consistently reliable**. This lack of stability, combined with a **loss probability of 4.4%**, makes Region 0 a **risky candidate** that fails to meet the defined risk criteria.

This indicates that, although profitable in many cases, Region 0 introduces too much variability to justify investment under strict risk constraints. Therefore, Region 0 is **not eligible** for final recommendation.


##### Region 1

In [ ]:
region_1_bs = bootstrap_profit(model = row_1['model'], features = row_1['x_valid'], targets = row_1['y_valid'])

In [ ]:
avg_profit_1 = np.mean(region_1_bs) # Average profit

conf_int_1 = np.percentile(region_1_bs, [2.5, 97.5]) # 95% confidence interval

loss_probability_1 = np.mean(np.array(region_1_bs) < 0) * 100 # Probability of loss if profit < 0

In [ ]:
print(f"Region 1 \nAverage Profit: {avg_profit_1:.2f}\nConfidence Interval: {conf_int_1}\nLoss Probabilty: {loss_probability_1}")

###### Region 1 Bootstrapping Summary

- **Average Profit:** 4,166,571.24  
- **95% Confidence Interval:** 524,696 to 8,124,816  
- **Probability of Loss:** 1.3% *(Below the 2.5% risk threshold)*

**Interpretation:**

Region 1 shows a strong and **consistent profit profile**, with an average net profit of approximately **4.17 million** per simulation. The **probability of loss is just 1.3%**, making it compliant with the company’s risk tolerance.

The **95% confidence interval** ranges from **524k to 8.12M**, which—although not extremely tight, is entirely **above zero**, indicating **a low likelihood of negative returns**. This range is significantly more stable compared to Region 0, reflecting **lower variability in outcomes** and stronger reliability of the model’s well selection.

Combined with its **very low RMSE (0.89)** and **minimal prediction error (1.29%)**, Region 1 is clearly the **most accurate and low-risk option** for investment.


##### Region 2

In [ ]:
region_2_bs = bootstrap_profit(model = row_2['model'], features = row_2['x_valid'], targets = row_2['y_valid'])

In [ ]:
avg_profit_2 = np.mean(region_2_bs) # Average profit

conf_int_2 = np.percentile(region_2_bs, [2.5, 97.5]) # 95% confidence interval

loss_probability_2 = np.mean(np.array(region_2_bs) < 0) * 100 # Probability of loss if profit < 0

In [ ]:
print(f"Region 2 \nAverage Profit: {avg_profit_2:.2f}\nConfidence Interval: {conf_int_2}\nLoss Probabilty: {loss_probability_2}")

###### Region 2 Bootstrapping Summary

- **Average Profit:** 3,117,091.44  
- **95% Confidence Interval:** -1,976,972 to 8,272,104  
- **Probability of Loss:** 13.1% *(Above the 2.5% risk threshold)*

**Interpretation:**

Although Region 2 offers the **highest average reserves (95k barrels)** across all regions, its **bootstrapped profit distribution reveals significant risk**. With a **13.1% chance of loss** and a confidence interval that dips nearly **2 million below zero**, Region 2 fails to meet the required risk tolerance.

The **wide and asymmetric confidence interval** suggests that well performance in Region 2 is **highly variable**, making outcomes less predictable. While some top-performing wells contribute to strong profits in certain samples, others fall far short, leading to a higher risk of negative return.

Given this volatility, Region 2 is **not recommended for development** under the current investment constraints.

In [ ]:
# Plot histogram of profits for all 3 regions
plt.figure(figsize=(12, 6))

plt.hist(region_0_bs, bins=50, alpha=0.6, label='Region 0', color='skyblue')
plt.hist(region_1_bs, bins=50, alpha=0.6, label='Region 1', color='lightgreen')
plt.hist(region_2_bs, bins=50, alpha=0.6, label='Region 2', color='salmon')

plt.title('Bootstrapped Profit Distribution by Region')
plt.xlabel('Profit ($)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.tight_layout()

## Risk and Confidence Final Summary

| Region   | Average Profit     | 95% Confidence Interval       | Loss Probability |
|----------|--------------------|-------------------------------|------------------|
| Region 0 | 4.37 million      | -656K to 9.37M              | **4.4%**       |
| Region 1 | 4.17 million      | 525K to 8.12M               | **1.3%**       |
| Region 2 | 3.12 million      | -1.98M to 8.27M             | **13.1%**      |

Only **Region 1** meets the risk criteria, with a loss probability below the **2.5% threshold** required for investment.

- **Region 1** stands out as the **only low-risk option**, offering consistent profits and the **lowest prediction error (1.29%)**.
- **Region 0** has similar average returns but a higher probability of loss and a confidence interval that includes negative values.
- **Region 2**, despite having the highest average reserves, is the **most volatile**, with a large spread in outcomes and the highest risk of financial loss.

# Final Recommendation

After evaluating all three regions across key business and modeling criteria, I recommend **developing Region 1**.

### Justification:

| Factor                  | Region 0               | Region 1               | Region 2               |
|-------------------------|------------------------|------------------------|------------------------|
| **Model Accuracy**      | RMSE = 37.66 (high error) | RMSE = 0.89 (lowest error) | RMSE = 40.08 (high error) |
| **Avg. Profit**         | 4.37M                 | 4.17M                 | 3.12M                 |
| **Risk (Loss %)**       | 4.4%                | 1.3%                | 13.1%               |
| **Confidence Interval** | -656K to 9.37M       | 525K to 8.12M        | -1.98M to 8.27M      |

### Summary:
- **Region 1** is the **only region** that meets the required **risk threshold** of less than 2.5%.
- It has the **lowest RMSE (0.89)** and **most accurate predictions**, enabling better selection of top-performing wells.
- Its **confidence interval is fully above zero**, reflecting more consistent returns than Region 0 and significantly less risk than Region 2.
- While Region 0 has slightly higher profit on average, it **exceeds the acceptable risk level** and includes a chance of loss.

### Business Decision:
> **Invest in Region 1.**  
> It delivers strong returns, meets the company’s risk policy, and offers the **best trade-off between profit and reliability**—critical for high-stakes oil well development.